In [1]:
import pandas as pd
import numpy as np
import xpress as xp # FICO Xprerss Solver
xp.init('/Applications/FICO Xpress/xpressmp/bin/xpauth.xpr')

from atcs import ATCS
import fig_plot as fp

In [ ]:
N_sample = 20
seed = 1
solver_type = 'minlp'
model_type = 'deterministic'
sample_type = f'head_{N_sample}'
data = ATCS(seed = 1)
data.choose_subset_point(N_sample, randomized = False) # Choose subset data

In [ ]:
location_fig = fp.create_l_fig()
location_fig = fp.add_point_to_l_fig(location_fig, data.l_df, size=5, name='All Data Points', color='blue')
location_fig = fp.add_point_to_l_fig(location_fig, data.l_sub_df, size=5, name='Sub Data Points', color='red')
location_fig.show()

In [4]:
data.r_sub_df.head()

,range
index,
120,127.887414
552,60.391929
90,130.136422
973,65.957089
667,41.008548


In [5]:
robot_loc = data.l_sub_df.to_numpy()

In [6]:
robot_loc

array([[-107.2610792 ,  -86.63810831],
       [ 132.1852957 ,  -77.49896536],
       [-108.7109858 ,  -83.91867376],
       [ -60.6773044 ,  -64.0680609 ],
       [ 145.5697713 ,  -84.81939248],
       [ -15.06147599,  -73.6672832 ],
       [-114.9544539 ,  -85.38126478],
       [-143.2967837 ,  -88.77250326],
       [-125.1596227 ,  -88.89849407],
       [-140.3191774 ,  -80.74077429],
       [ -68.71749331,  -67.67320423],
       [-127.6477275 ,  -87.3478946 ],
       [-122.1782837 ,  -87.62479045],
       [-152.2635181 ,  -84.23190802],
       [ -32.94573352,  -77.49056887],
       [ 149.5781947 ,  -82.65163704],
       [ 151.4398842 ,  -83.51043419],
       [ 171.4268413 ,  -85.70132796],
       [ -48.76111705,  -80.5262454 ],
       [ 151.1457948 ,  -89.78722279]])

In [ ]:
# MINLP Model
# Data
robot_id = data.r_sub_df.index.to_numpy()
robot_loc = data.l_sub_df.to_numpy()
robot_range = data.r_sub_df.to_numpy().flatten()
x_min,x_max = np.min(robot_loc, axis=0)[0], np.max(robot_loc, axis=0)[0]
y_min,y_max = np.min(robot_loc, axis=0)[1], np.max(robot_loc, axis=0)[1]
dist_matrix = data.get_distance_matrix(sample_subset = True)

# Creates an empty Model
model = xp.problem(name='MINLP_Deterministic')

# Given Inputs and Parameters
R = data.r_sub_df['range'].to_dict()
L_Range = data.r_min # km
U_Range = data.r_max # km
U_Robot = data.q # Max robots per charger
U_Charger = data.m # Max chargers per station
L_Station = int(np.ceil(N_sample/(U_Charger * U_Robot))) # Maximum Robots per opened station
Dist_Max = np.sqrt((x_max - x_min)**2 + (y_max - y_min)**2)
C_b = data.c_b # Investment cost per station
C_h = data.c_h # Cost of moving a robot
C_m = data.c_m # Maintenance cost per charger
C_c = data.c_c # Charging cost per km


Long_V, Lat_V = data.l_sub_df['longitude'].to_dict(), data.l_sub_df['latitude'].to_dict()

# Parameters
# U_Station = L_Station + 1 # For now
U_Station = len(robot_id)
data.set_output_folder(solver_type, model_type, name = f'{N_sample}_{sample_type}samp_{U_Station}s_seed{seed}')

# U_Station = N_sample
# Sets and Notation
V = data.r_sub_df.index.to_numpy() # Robot Vehicle sets
S = np.arange(U_Station)


# Output Variables
x = model.addVariables(S, name = 'x', vartype = xp.continuous, lb = x_min, ub = x_max) # Longitude of station s
y = model.addVariables(S, name = 'y', vartype = xp.continuous, lb = y_min, ub = y_max) # Latitude of station s

eta = model.addVariables(S, name = 'eta', vartype = xp.integer) # Number of charger installed at station s
mu = model.addVariables(S, name = 'mu', vartype = xp.binary) # Whether station s is built
beta = model.addVariables(V, S, name = 'beta', vartype = xp.binary) # Whether robot v is covered by station s
alpha = model.addVariables(V, S, name = 'alpha', vartype = xp.binary) # Whether robot v is brought to station s by taking penalty
d = model.addVariables(V, S, name = 'd', vartype = xp.continuous, lb = 0) # Distance from v to s


# Constraints
# Robot v can be allocated to charging station s iff it is built
model.addConstraint(beta[v,s] <= mu[s] for v in V for s in S) 

# Each Robot v must be allocated to only one station
model.addConstraint(xp.Sum(beta[v,s] for s in S) == 1 for v in V)

# Number of chargers per station
model.addConstraint(eta[s] >= xp.Sum(beta[v,s] for v in V)/U_Robot for s in S)
model.addConstraint(eta[s] <= U_Charger for s in S)
# Robot v can be brought to station s by human iff the robot is allocated to that station
model.addConstraint(alpha[v,s] <= beta[v,s] for v in V for s in S)

# Distance from robot v to station s
# If Robot v is allocated to station s,
#       If its range can reach s, alpha = 0
#       Else; distance is 0 and it is brought to s by taking penalty (alpha = 1)
# Else; d[v,s] = 0
model.addConstraint(d[v,s] <= R[v] * (1 - alpha[v,s]) for v in V for s in S)
model.addConstraint(d[v,s] >= xp.sqrt((Long_V[v] - x[s])**2 + (Lat_V[v] - y[s])**2)
                                - Dist_Max * (alpha[v,s] + (1 - beta[v,s])) 
                                        for v in V for s in S)
# model.addConstraint(d[v,s]**2 >= (Long_V[v] - x[s])**2 + (Lat_V[v] - y[s])**2
#                                 - (np.max(dist_matrix)**2) * (alpha[v,s] + (1 - beta[v,s])) 
#                                         for v in V for s in S)
# model.addConstraint(d[v,s] >= (Long_V[v] - x[s])**2 + (Lat_V[v] - y[s])**2
#                                 - np.max(dist_matrix) * (alpha[v,s] + (1 - beta[v,s])) 
#                                         for v in V for s in S)
model.addConstraint(d[v,s] <= Dist_Max * beta[v,s] for v in V for s in S)
# model.addConstraint(alpha[120,0] == 1)
# model.addConstraint(d[v,s] <= np.min(dist_matrix[dist_matrix != 0]) * (1 - alpha[v,s]) for v in V for s in S)
# model.addConstraint(d[v,s] >= np.min(dist_matrix[dist_matrix != 0]) - np.max(dist_matrix) * (alpha[v,s] + (1 - beta[v,s])) 
#                                     for v in V for s in S)
# model.addConstraint(d[v,s] <= np.max(dist_matrix)*beta[v,s] for v in V for s in S)

# Objective Function
Build_station_cost = xp.Sum(C_b * mu[s] for s in S)
Build_charger_cost = xp.Sum(C_m * eta[s] for s in S)
Penalty_cost = xp.Sum(C_h * alpha[v,s] for v in V for s in S)
Charging_cost = xp.Sum(C_c * (beta[v,s]*U_Range - (beta[v,s]*R[v] - d[v,s])) for v in V for s in S)

obj = Build_station_cost + Build_charger_cost + Charging_cost + Penalty_cost

model.setObjective(obj, sense=xp.minimize)
model.setControl('miprelstop', 1e-3)
model.setControl('maxtime', 3600)
# Solve the problem
model.solve()

Original problem size
   linear:    2060 rows, 1280 columns, 4440 linear coefficients
   nonlinear: 400 coefficients, 12400 tokens
             2800 mul             0 div       400 sqrt
Nonlinear presolve
   linear presolve removed 420 rows, 0 columns, 420 linear coefficients
   creating 800 '+' clusters removed 800 tokens
Presolved problem size
   linear:    1640 rows, 1281 columns, 4020 linear coefficients
   nonlinear: 400 coefficients, 11600 tokens
             2000 mul             0 div       400 sqrt
                0 exp             0 log       800 pow
Problem is nonlinear presolved
FICO Xpress v9.4.2, Hyper, solve started 10:29:22, Mar 28, 2025
Control settings used:
MAXTIME = 3600
OUTPUTLOG = 1
MIPRELSTOP = .001
NLPPOSTSOLVE = 1
XSLP_DELETIONCONTROL = 0
XSLP_OBJSENSE = 1
Maximum expanded nl-formula size: 29  (row 'R1261')
Total tokens: 11600
  11  parallel calculation threads
  Jacobian: symbolic differentiation
         400 base AD formula, 24 average complexity
        1600 

(<SolveStatus.STOPPED: 1>, <SolStatus.FEASIBLE: 2>)

In [8]:
np.min(dist_matrix[dist_matrix != 0])

2.0502244607013913

In [9]:
model.getSolution(obj)

17996.466485225395

In [10]:
mu_dict = model.getSolution(mu)
eta_dict = model.getSolution(eta)
x_dict = model.getSolution(x)
y_dict = model.getSolution(y)

station_df = pd.DataFrame([mu_dict, eta_dict, x_dict, y_dict], ['mu','eta','longitude','latitude']).T
station_df['Building_Station_Cost'] = station_df['mu'] * C_b
station_df['Building_Charger_Cost'] = station_df['eta'] * C_m

In [11]:
beta_dict = model.getSolution(beta)
alpha_dict = model.getSolution(alpha)
d_dict = model.getSolution(d)

robot_df = pd.DataFrame([beta_dict, alpha_dict, d_dict],['beta','alpha','d']).T.reset_index()
robot_df.rename(columns={'index':'(v,s)'}, inplace=True)
robot_df['v'], robot_df['s'] = robot_df['(v,s)'].apply(lambda x: x[0]), robot_df['(v,s)'].apply(lambda x: x[1])
robot_df.drop(['(v,s)'], axis = 1, inplace= True)
robot_df = robot_df[['v','s','beta','alpha','d']]
# robot_df = robot_df.groupby(['s','v']).sum()

In [12]:
model.getSolution(Charging_cost)

996.4664852253925

In [13]:
robot_df[robot_df['beta']==1]

,v,s,beta,alpha,d
5,120,5,1.0,0.0,8.495051
36,552,16,1.0,0.0,18.706966
45,90,5,1.0,0.0,7.085208
65,973,5,1.0,0.0,58.944856
96,667,16,1.0,0.0,4.464072
116,314,16,1.0,1.0,0.000000
125,181,5,1.0,0.0,0.702091
145,119,5,1.0,0.0,27.855978
165,49,5,1.0,0.0,10.153451
185,101,5,1.0,0.0,25.087634


In [14]:
station_df

,mu,eta,longitude,latitude,Building_Station_Cost,Building_Charger_Cost
0,0.0,0.0,-152.263518,-89.787223,0.0,0.0
1,0.0,0.0,-152.263518,-89.787223,0.0,0.0
2,0.0,0.0,-152.263518,-89.787223,0.0,0.0
3,0.0,0.0,-152.263518,-89.787223,0.0,0.0
4,0.0,0.0,-152.263518,-89.787223,0.0,0.0
5,1.0,6.0,-115.654550,-85.328317,5000.0,3000.0
6,0.0,0.0,-152.263518,-89.787223,0.0,0.0
7,0.0,0.0,-152.263518,-89.787223,0.0,0.0
8,0.0,0.0,-152.263518,-89.787223,0.0,0.0
9,0.0,0.0,-152.263518,-89.787223,0.0,0.0


In [16]:
robot_df.to_csv('minlp_output/deterministic/20_samp_20s_seed1/robot_df.csv')
station_df.to_csv('minlp_output/deterministic/20_samp_20s_seed1/station_df.csv')

In [17]:
heu_station_df = pd.DataFrame( [(-120.28422860594455, -85.14822308962202), (-43.75665270724203, -70.98962125892272), (150.21514729388687, -83.492662445044)])
heu_station_df.columns = ['longitude','latitude']

In [18]:
heu_station_df

,longitude,latitude
0,-120.284229,-85.148223
1,-43.756653,-70.989621
2,150.215147,-83.492662


In [19]:
location_fig = fp.create_l_fig()
# location_fig = fp.add_point_to_l_fig(location_fig, data.l_df, size=5, name='All Data Points', color='blue')
location_fig = fp.add_point_to_l_fig(location_fig, data.l_sub_df, size=5, name='Sub Data Points', color='red')
location_fig = fp.add_point_to_l_fig(location_fig, station_df[station_df['mu'] != 0], size=7, name='MINLP Stations', color='green')
location_fig = fp.add_point_to_l_fig(location_fig, heu_station_df, size=7, name='Heuristic Stations', color='Brown')
location_fig.show()